<a href="https://colab.research.google.com/github/mostume222/inf-test/blob/master/infa_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Step 0.** If for some reason you encounter issues navigating the folders in the project or having errors you can delete the project folder in the online colab environment by using the following code cell. If you are just starting to use this fresh colab notebook skip this step.

In [18]:
!cd ~
!pwd
!ls
!rm -r inf-test/

/content
inf-test  sample_data


**Step 1.** Clone github repository from link https://github.com/mostume222/inf-test.git , this will download the source code and also make the correct project structure to tun the compression program.

In [19]:
!git clone https://github.com/mostume222/inf-test.git
%cd inf-test
!git checkout master #go to master branch to have the correct project structure

Cloning into 'inf-test'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 110 (delta 30), reused 79 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (110/110), 16.07 MiB | 11.13 MiB/s, done.
Resolving deltas: 100% (30/30), done.
/content/inf-test
Branch 'master' set up to track remote branch 'master' from 'origin'.
Switched to a new branch 'master'


**Step 2.** Install the conda environment in /inf-test/env/infa.yml to install dependencies neded to run the compression procedure.

If you would like to install your own dependencies, the important ones are:

* cd-hit 4.8.1  
* bowtie2 2.5.4
* cutadapt 3.5
* python 3.6.13
* perl 5.34.0
* pip 21.2.2

In [2]:
#install condacolab
!pip install -q condacolab
import condacolab
condacolab.install()

#create conda envionment in infa.yml file
!conda env create -f content/infa-test/env/infa.yml --quiet

✨🍰✨ Everything looks OK!

EnvironmentFileNotFound: '/content/content/infa-test/env/infa.yml' file not found



In [ ]:
!ls

condacolab_install.log	data  env  infa_main.ipynb  output_compression	README.md  sequences  src


In [ ]:
!pwd
!rm -r output_compression/1,.comp/

/content/inf-test
rm: cannot remove 'output_compression/1,.comp/': No such file or directory


In [ ]:
!git checkout master
!ls

D	output_compression/1,.comp/acce.list
D	output_compression/1,.comp/acce_variants.family
D	output_compression/1,.comp/andy
D	output_compression/1,.comp/cluster.coor
D	output_compression/1,.comp/cluster.coors
D	output_compression/1,.comp/cluster.coors_sorted
D	output_compression/1,.comp/cluster_size
D	output_compression/1,.comp/database.compressed
D	output_compression/1,.comp/database.fasta
D	output_compression/1,.comp/domain.list
D	output_compression/1,.comp/hash.coors
D	output_compression/1,.comp/kmers_clean.fasta
D	output_compression/1,.comp/kmers_w.fasta
D	output_compression/1,.comp/result
D	output_compression/1,.comp/result.clstr
D	output_compression/1,.comp/rp
D	output_compression/1,.comp/taxids_hash.dmp
D	output_compression/1,.comp/unique.list
D	output_compression/1,.comp/unique.positions
Already on 'master'
Your branch is up to date with 'origin/master'.
condacolab_install.log	data  env  infa_main.ipynb  output_compression	README.md  sequences  src


In [ ]:
!ls env

test.yml


In [ ]:
## install conda in colab
!pip install -q condacolab
import condacolab
condacolab.install()


✨🍰✨ Everything looks OK!


In [ ]:
## create conda environment
!conda env create -f env/test.yml

Channels:
 - defaults
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.7.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



python-3.6.13        | 32.5 MB   | :   0% 0/1 [00:00<?, ?it/s]
bowtie2-2.5.4        | 14.3 MB   | :   0% 0/1 [00:00<?, ?it/s]

perl-5.34.0          | 12.4 MB   | :   0% 0/1 [00:00<?, ?it/s]


openssl-1.1.1w       | 3.7 MB    | :   0% 0/1 [00:00<?, ?it/s]



libstdcxx-ng-14.1.0  | 3.7 MB    | :   0% 0/1 [00:00<?, ?it/s]




tk-8.6.14            | 3.4 MB    | :   0% 0/1 [00:00<?, ?it/s]





pip-21.2.2           | 1.8 MB    | :   0% 0/1 [00:00<?, ?it/s]






sqlite-3.45.3        | 1.2 MB    | :   0% 0/1 [00:00<?, ?it/s]







ncurses-6.4          | 914 KB    | :   0% 0/1 [00:00<?, ?it/s]








libgcc-ng-14.1.0     | 822 KB    | :   0% 0/1 [00:00<?, ?it/s]









setuptools-58.0.4    | 788 KB  

In [ ]:
## check cd hit instalation
!source activate infc && cd-hit --version

		====== CD-HIT version 4.8.1 (built on Apr 11 2024) ======

Usage: cd-hit [Options] 

Options

   -i	input filename in fasta format, required, can be in .gz format
   -o	output filename, required
   -c	sequence identity threshold, default 0.9
 	this is the default cd-hit's "global sequence identity" calculated as:
 	number of identical amino acids or bases in alignment
 	divided by the full length of the shorter sequence
   -G	use global sequence identity, default 1
 	if set to 0, then use local sequence identity, calculated as :
 	number of identical amino acids or bases in alignment
 	divided by the length of the alignment
 	NOTE!!! don't use -G 0 unless you use alignment coverage controls
 	see options -aL, -AL, -aS, -AS
   -b	band_width of alignment, default 20
   -M	memory limit (in MB) for the program, default 800; 0 for unlimitted;
   -T	number of threads, default 1; with 0, all CPUs will be used
   -n	word_length, default 5, see user's guide for choosing it
   -l	length of thr

In [ ]:
# give exp permissions to src run
%cd src/compressor/
!source activate infc && ls


/content/inf-test/src/compressor
cluster_acc_family.pl	    demo		       lucas_compressor_b.sh
cluster_acc.pl		    divider.pl		       lucas_compressor.sh
cluster_acc_variant.pl	    enumerate.pl	       remove.sh
cluster_element_counter.pl  enumerate.py	       run_compress.sh
concat_clusters.sh	    enumerate.sh	       uniques_concatenation.pl
concat_clusters_test.sh     kmer.pl		       uniques_entanglement.pl
counter.pl		    label_clusters_variant.sh  uniques_gitrog.pl
count_lenghts_pieces.pl     lucas_back		       uniques_intanglement.pl


In [ ]:
!chmod +x run_compress.sh
!source activate infc && ls

cluster_acc_family.pl	    demo		       lucas_compressor_b.sh
cluster_acc.pl		    divider.pl		       lucas_compressor.sh
cluster_acc_variant.pl	    enumerate.pl	       remove.sh
cluster_element_counter.pl  enumerate.py	       run_compress.sh
concat_clusters.sh	    enumerate.sh	       uniques_concatenation.pl
concat_clusters_test.sh     kmer.pl		       uniques_entanglement.pl
counter.pl		    label_clusters_variant.sh  uniques_gitrog.pl
count_lenghts_pieces.pl     lucas_back		       uniques_intanglement.pl


In [ ]:
#  run the script with conda env
!source activate infc && ./run_compress.sh 0.94 0.958 200 50 100000 1,.comp


Streaming output truncated to the last 5000 lines.
current genome kmer 8668 20
current genome kmer 8669 0
bive: 8668_19 20
to  : 8668 950 250
current genome kmer 8669 4
bive: 8669_0 0
to  : 8669 0 200
current genome kmer 8669 12
bive: 8669_4 4
to  : 8669 200 200
current genome kmer 8669 13
current genome kmer 8669 14
current genome kmer 8669 15
current genome kmer 8669 16
current genome kmer 8669 17
current genome kmer 8669 18
current genome kmer 8669 19
current genome kmer 8669 20
current genome kmer 8671 3
bive: 8669_12 20
to  : 8669 600 600
current genome kmer 8672 7
bive: 8671_3 3
to  : 8671 150 200
current genome kmer 8672 17
bive: 8672_7 7
to  : 8672 350 200
current genome kmer 8672 18
current genome kmer 8673 0
bive: 8672_17 18
to  : 8672 850 250
current genome kmer 8673 17
bive: 8673_0 0
to  : 8673 0 200
current genome kmer 8673 18
current genome kmer 8673 19
current genome kmer 8673 20
current genome kmer 8675 0
bive: 8673_17 20
to  : 8673 850 350
current genome kmer 8675 10
b

In [ ]:
#clean
!rm -r /content/inf-test/output_compression/1,.comp/

rm: cannot remove '/content/inf-test/output_compression/1,.comp/': No such file or directory
